# Ejercicios SQL

In [4]:
# Import SparkSession
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder \
        .master("local[1]") \
        .appName("Ejercicios SQL") \
        .getOrCreate()

In [5]:
# Import data from agents.csv
df_agents = spark.read.option("header",True) \
        .option("inferSchema",True) \
        .csv("./data/agents.csv", )
df_agents.printSchema()
df_agents.show()

root
 |-- agentid: integer (nullable = true)
 |-- name: string (nullable = true)

+-------+------------------+
|agentid|              name|
+-------+------------------+
|      0|  Michele Williams|
|      1|    Jocelyn Parker|
|      2|Christopher Moreno|
|      3|       Todd Morrow|
|      4|       Randy Moore|
|      5|        Paul Nunez|
|      6|      Gloria Singh|
|      7|      Angel Briggs|
|      8|      Lisa Cordova|
|      9|        Dana Hardy|
|     10|           Agent X|
+-------+------------------+



In [2]:
# Import data from calls.csv
df_calls = spark.read.option("header",True) \
        .option("inferSchema",True) \
        .csv("./data/calls.csv", )
df_calls.printSchema()
df_calls.show()
df_calls.count()

root
 |-- callid: integer (nullable = true)
 |-- agentid: integer (nullable = true)
 |-- customerid: integer (nullable = true)
 |-- pickedup: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- productsold: integer (nullable = true)

+------+-------+----------+--------+--------+-----------+
|callid|agentid|customerid|pickedup|duration|productsold|
+------+-------+----------+--------+--------+-----------+
|     0|     10|       179|       0|       0|          0|
|     1|      5|       691|       1|     116|          0|
|     2|     10|        80|       1|     165|          0|
|     3|      6|       629|       1|     128|          0|
|     4|      8|       318|       1|     205|          0|
|     5|      7|       319|       1|     225|          1|
|     6|     10|       265|       1|     211|          0|
|     7|      9|       625|       0|       0|          0|
|     8|      5|       877|       0|       0|          0|
|     9|      5|       191|       1|     145|     

9940

In [13]:
# Import data from customers.csv
df_customers = spark.read.option("header",True) \
        .option("inferSchema",True) \
        .csv("./data/customers.csv", )
df_customers.printSchema()
df_customers.show()

root
 |-- customerid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- email: string (nullable = true)
 |-- company: string (nullable = true)
 |-- phonenumber: string (nullable = true)
 |-- Age: integer (nullable = true)

+----------+------------------+--------------------+--------------------+--------------------+------------+---+
|customerid|              name|          occupation|               email|             company| phonenumber|Age|
+----------+------------------+--------------------+--------------------+--------------------+------------+---+
|         0|      David Melton|          Unemployed|    DMelton@zoho.com|Morris, Winters a...|409-093-0748| 16|
|         1|  Michael Gonzalez|             Student|Gonzalez_Michael@...|  Hernandez and Sons|231-845-0673| 19|
|         2|     Amanda Wilson|             Student|Amanda.Wilson75@v...|Mooney, West and ...|844-276-4552| 18|
|         3|     Robert Thomas|Engineer, struc

1000

In [18]:
# Create temporary tables
df_agents.createOrReplaceTempView("agents")       # Agentes
df_calls.createOrReplaceTempView("calls")         # Llamadas
df_customers.createOrReplaceTempView("customers") # Clientes

## Ejercicio 1

Extraer agentes cuyo nombre empiezen por M o terminen en O

In [ ]:
# SQL Select query
df_ej_1 = spark.sql("""
select * from agents
where name like 'M%' or name like '%o'
""")
df_ej_1.show()

## Ejercicio 2
Escriba una consulta que produzca una lista, en orden alfabético,  de todas las distintas ocupaciones en la tabla Customer que contengan la palabra "Engineer".

In [ ]:
df_ej_2 = spark.sql("""
SELECT DISTINCT Occupation
FROM customers
WHERE Occupation LIKE '%Engineer%'
ORDER BY Occupation
""")
df_ej_2.show()

## Ejercicio 3

Escriba una consulta que devuelva el ID del cliente, su nombre y una columna  Mayor30 que contenga "Sí" si el cliente tiene más de 30 años y "No" en caso contrario.

In [ ]:
df_ej_3 = spark.sql("""
SELECT CustomerID, Name,
    CASE
        WHEN Age >= 30 THEN 'Yes'
        WHEN Age <  30 THEN 'No'
        ELSE 'Missing Data'
    END AS Over30
FROM customers
ORDER BY Name DESC
""")
df_ej_3.show()

## Ejercicio 4

Escriba una consulta que devuelva todas las llamadas realizadas a clientes de la  profesión de ingeniería y muestre si son mayores o menores de 30, así como si  terminaron comprando el producto de esa llamada.

In [ ]:
df_ej_4 = spark.sql("""
SELECT CallID, Cu.CustomerID, Name, ProductSold,
    CASE
        WHEN Age >= 30 THEN 'Yes'
        WHEN Age <  30 THEN 'No'
        ELSE 'Missing Data'
    END AS Over30
FROM customers Cu
JOIN calls Ca ON Ca.CustomerID = Cu.CustomerID
WHERE Occupation LIKE '%Engineer%'
ORDER BY Name DESC
""")
df_ej_4.show()

## Ejercicio 5

Escriba dos consultas: una que calcule las ventas totales y las llamadas totales realizadas a los clientes de la profesión de ingeniería y otra que calcule las mismas métricas para toda la base de clientes

In [ ]:
df_ej_5 = spark.sql("""
SELECT SUM(ProductSold) AS TotalSales, COUNT(*) AS NCalls
FROM customers Cu
JOIN calls Ca ON Ca.CustomerID = Cu.CustomerID
WHERE Occupation LIKE '%Engineer%'
""")
df_ej_5.show()

## Ejercicio 6

Escriba una consulta que devuelva, para cada agente, el nombre del agente, la cantidad de llamadas, las llamadas más largas y más cortas, la duración promedio de las llamadas y la cantidad total de productos vendidos. Nombra las columnas AgentName, NCalls, Shortest, Longest, AvgDuration y TotalSales.

Luego ordena la tabla por AgentName en orden alfabético.  (Asegúrese de incluir la cláusula WHERE PickedUp = 1 para calcular solo el promedio de todas las llamadas que fueron atendidas (de lo contrario, ¡todas las duraciones mínimas serán 0)!)

In [ ]:
df_ej_6 = spark.sql("""
SELECT Name AS AgentName, COUNT(*) AS NCalls, MIN(Duration) AS Shortest, MAX(Duration) AS Longest, ROUND(AVG(Duration),2) AS AvgDuration, SUM(ProductSold) AS TotalSales
FROM calls C
    JOIN agents A ON C.AgentID = A.AgentID
WHERE PickeDup = 1
GROUP BY Name
ORDER BY Name
""")
df_ej_6.show()

## Ejercicio 7

Dos métricas del desempeño de los agentes de ventas que le interesan a su empresa son: 
1. para cada agente, cuántos segundos en promedio les toma vender un producto cuando tienen éxito
2. para cada agente, cuántos segundos en promedio permanecen en el teléfono antes de darse por vencidos cuando no tienen éxito. Escribe una consulta que calcule esto

In [ ]:
df_ej_7 = spark.sql("""
SELECT a.name,
SUM(
   CASE
       WHEN productsold = 0 THEN duration
       ELSE 0
   END)/SUM(
   CASE
       WHEN productsold = 0 THEN 1
       ELSE 0
   END)
AS avgWhenNotSold ,
SUM(
   CASE
       WHEN productsold = 1 THEN duration
       ELSE 0
   END)/SUM(
       CASE WHEN productsold = 1 THEN 1
       ELSE 0
   END)
AS avgWhenSold
FROM calls c
JOIN agents a ON c.agentid = a.agentid
GROUP BY a.name
ORDER BY 1
""")
df_ej_7.show()